In [1]:
import numpy as np
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import datetime

In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2021-10-15 23:56:21.066674: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-15 23:56:21.076509: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-15 23:56:21.077133: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
IMPORT_COUNT = 5000000
TEST_COUNT = 10000

In [4]:
# Generate random seed
#myrand=np.random.randint(1, 99999 + 1)
myrand=71926
np.random.seed(myrand)
tf.random.set_seed(myrand)
print("Random seed is:",myrand)

Random seed is: 71926


In [5]:
RNG_OUTPUT_FILENAME="mersenne_twist_xorshifter.txt"
df = np.genfromtxt(RNG_OUTPUT_FILENAME,delimiter=', ',dtype='uint64', max_rows=IMPORT_COUNT)

In [6]:
#calculates how many bits are in the output.
BIT_WIDTH=np.ceil(np.log2(np.amax(df))).astype(int)
# convert the generated numbers to binary sequences
df_as_bits = np.concatenate(((df[:, 0][:,None] & (1 << np.arange(BIT_WIDTH,dtype='uint64')) > 0).astype(int), (df[:, 1][:,None] & (1 << np.arange(BIT_WIDTH,dtype='uint64')) > 0).astype(int)), axis=1)

In [7]:
# shuffle the data frames
indicies = np.arange(df_as_bits.shape[0],dtype='uint64')
np.random.shuffle(indicies)
df_as_bits=df_as_bits[indicies]

In [8]:
# convert the data into inputs and outputs
y = df_as_bits[:,32:]
X = df_as_bits[:,:32]

In [9]:
# Split the data into train and test data
X_train = X[TEST_COUNT:]
X_test = X[:TEST_COUNT]
y_train = y[TEST_COUNT:]
y_test = y[:TEST_COUNT]

In [10]:
def build_model(hp):
    LOSS="binary_crossentropy"
    model = Sequential()
    model.add(Dense(640, activation='relu',input_shape=[X.shape[1]] ))
    model.add(Dense(y.shape[1], activation='sigmoid'))
    
    opt = keras.optimizers.Nadam(
        learning_rate=hp.Float("learning_rate", 10**(-5), 10**(-2),sampling="log"),
        epsilon=hp.Float("epsilon",1e-7,1e-4,sampling="log"),
        beta_1=hp.Float("beta_1",.8,.99999,sampling="reverse_log"),
        beta_2=hp.Float("beta_2",.8,.99999,sampling="reverse_log"),
        )
    model.compile(optimizer=opt, loss=LOSS,metrics=['binary_accuracy'])
    model.summary()
    return model

In [11]:
#define callback functions
stopEarly = tf.keras.callbacks.EarlyStopping(
    monitor='val_binary_accuracy', min_delta=.001, patience=10, verbose=0, mode='auto', restore_best_weights=False
)

log_dir = "tmp/hyperparameters_tempering/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,profile_batch=0)

In [12]:
# extract a short set from the training for hyper parameter tuning
X_train_short= X_train[:1000000]
y_train_short= y_train[:1000000]

In [13]:
%%time
# Tuning the hyper parameters
tuner = kt.tuners.bayesian.BayesianOptimization(build_model,'val_binary_accuracy',25,project_name="tmp/tempering", seed=myrand)

while tuner.remaining_trials>0:
    try:
        tuner.search(X_train_short, y_train_short,batch_size=2048, epochs=100, validation_data=(X_test,y_test),callbacks=[stopEarly,tensorboard_callback])
    except:
        pass

tuner.results_summary()

Trial 25 Complete [00h 02m 18s]
val_binary_accuracy: 0.7568156123161316

Best val_binary_accuracy So Far: 0.9998593926429749
Total elapsed time: 00h 43m 12s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in ./tmp/tempering
Showing 10 best trials
Objective(name='val_binary_accuracy', direction='max')
Trial summary
Hyperparameters:
learning_rate: 0.01
epsilon: 0.0001
beta_1: 0.8000000000000002
beta_2: 0.8000000000000002
Score: 0.9998593926429749
Trial summary
Hyperparameters:
learning_rate: 0.01
epsilon: 0.0001
beta_1: 0.8810412165270334
beta_2: 0.8000000000000002
Score: 0.9998406171798706
Trial summary
Hyperparameters:
learning_rate: 0.01
epsilon: 0.0001
beta_1: 0.9031492462258401
beta_2: 0.9021489810373363
Score: 0.9995343685150146
Trial summary
Hyperparameters:
learning_rate: 0.01
epsilon: 0.0001
beta_1: 0.8538783427826274
beta_2: 0.8512431397642117
Score: 0.9845343828201294
Trial summary
Hyperparameters:
learning_rate: 0.01
epsilon: 3.1636418824271017e-06
beta_1: 0.800

In [14]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
# use the best model for training
model = tuner.hypermodel.build(best_hps)

best_hps.values

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 640)               21120     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                20512     
Total params: 41,632
Trainable params: 41,632
Non-trainable params: 0
_________________________________________________________________


{'learning_rate': 0.01,
 'epsilon': 0.0001,
 'beta_1': 0.8000000000000002,
 'beta_2': 0.8000000000000002}

In [15]:
def train_model(model, X = X_train, Y=y_train, epochs=100, batch_size=2048,verbose=1, log_dir = "tmp/dense_model/"):
    log_dir += datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1,profile_batch=0)
    model.fit(X, Y, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size,callbacks=[tensorboard_callback],verbose=verbose)
    return model

In [16]:
%%time
my_model_trained = train_model(model, epochs=30,log_dir = "tmp/mt_tempering/")

Epoch 1/30
2437/2437 [==============================] - 7s 3ms/step - loss: 0.4978 - binary_accuracy: 0.6624 - val_loss: 0.3570 - val_binary_accuracy: 0.7718
Epoch 2/30
2437/2437 [==============================] - 7s 3ms/step - loss: 0.2782 - binary_accuracy: 0.8192 - val_loss: 0.2282 - val_binary_accuracy: 0.8524
Epoch 3/30
2437/2437 [==============================] - 7s 3ms/step - loss: 0.1687 - binary_accuracy: 0.9019 - val_loss: 0.1136 - val_binary_accuracy: 0.9372
Epoch 4/30
2437/2437 [==============================] - 7s 3ms/step - loss: 0.0890 - binary_accuracy: 0.9470 - val_loss: 0.0756 - val_binary_accuracy: 0.9517
Epoch 5/30
2437/2437 [==============================] - 7s 3ms/step - loss: 0.0673 - binary_accuracy: 0.9567 - val_loss: 0.0559 - val_binary_accuracy: 0.9656
Epoch 6/30
2437/2437 [==============================] - 7s 3ms/step - loss: 0.0514 - binary_accuracy: 0.9670 - val_loss: 0.0483 - val_binary_accuracy: 0.9677
Epoch 7/30
2437/2437 [==============================

In [17]:
results = model.evaluate(X_test, y_test, batch_size=128)
print("test loss: %f, test acc: %s" % tuple(results))

79/79 [==============================] - 0s 1ms/step - loss: 6.3016e-05 - binary_accuracy: 1.0000
test loss: 0.000063, test acc: 1.0


In [18]:
model.save("mt_tempering_model.h5")